In [1]:
from fonctions import *
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd
import numpy as np 
import openpyxl
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
import statsmodels.api as sm
import os
from Fbref_alex.fonctions_preprocess import *
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


In [17]:
df_2324, affiches  = fonction_resultats(2023)
first = affiches['Wk'].min()
df_2324 = df_2324[df_2324['Journée'] <= first]
df_2324 = fonction_prepa_base(df_2324)
df_pred = pd.read_csv('dataframe_final.csv',encoding = 'utf-8')
df_pred = pd.concat([df_pred,df_2324[df_2324['Journée'] <= first-1]])
df_pred['Historique'] = df_pred.groupby(['Equipe 1', 'Equipe 2'])['Résultat'].cumsum() - df_pred['Résultat']



In [30]:
df_stats = pd.read_csv('Fbref_alex\\SOCCER_241223.csv')
mapping_equipe = {
        'Nimes': 'Nîmes',
        'Paris S-G': 'PSG',
        'Saint-Étienne': 'ASSE'
    }

df_stats = renommer_colonnes(preprocess_intitial(df_stats, mapping_equipe))
df_stats[['GF', 'GA']] = df_stats[['GF', 'GA']].astype(float).astype(int)
df_stats.insert(0, 'Victoire',  df_stats['Result'].apply(lambda x: 1 if x == 'W' else 0))
df_stats.insert(1, 'Défaite', df_stats['Result'].apply(lambda x: 1 if x == 'L' else 0))
df_stats.insert(2, 'Nul', df_stats['Result'].apply(lambda x: 1 if x == 'D' else 0))
df_stats.drop(['Result'], axis=1, inplace=True)

def fonction_accueil(widget_to_destroy, bouton_accueil):
    for widget in widget_to_destroy:
        widget.destroy()
    bouton_accueil.destroy()
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_intro.place(in_=window, anchor="w", relx=0.47, rely=0.1)

    label_choix = tk.Label(frame_menus, text="Que voulez vous faire?")
    label_choix.grid(row=0, column=0, padx=10, pady=10)

    choix_combobox = ttk.Combobox(frame_menus, values=['Visualisez des stats', 'Parcourir les données', 'Prédire un résultat futur'], state="readonly")
    choix_combobox.set("Sélectionnez un choix")
    choix_combobox.grid(row=0, column=1, padx=10, pady=10)
    choix_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix(label_choix, choix_combobox, e))

    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)


def fonction_choix(label_choix, choix_combobox, event):
    sélection = choix_combobox.get()
    if sélection == 'Visualisez des stats':
        fonction_choix_saison(label_choix, choix_combobox, event = None)
    elif sélection == 'Prédire un résultat futur':
        fonction_predictions(label_choix, choix_combobox, event = None)
    else:
        fonction_stats(label_choix, choix_combobox, event = None)

def fonction_choix_saison(label_choix, choix_combobox, event):
    label_choix.destroy()
    choix_combobox.destroy()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)
    label_choix_saison = tk.Label(frame_menus, text="Pour quelle saison?")
    label_choix_saison.grid(row=0, column=0, padx=10, pady=10)

    choix_saisons = df_stats['Saison'].unique().tolist()
    choix_saisons =  choix_saisons[:] + ['Toutes les saisons'] 
    saison_combobox = ttk.Combobox(frame_menus, values=choix_saisons, state="readonly")
    saison_combobox.set("Sélectionnez une saison")
    saison_combobox.grid(row=0, column=1, padx=10, pady=10)
    saison_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_equipe(label_choix_saison, saison_combobox, e))


def fonction_choix_equipe(label_choix_saison, saison_combobox, e):
    saison = saison_combobox.get()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.32)

    label_choix_equipe = tk.Label(frame_menus, text="Pour quelle équipe?")
    label_choix_equipe.grid(row=0, column=0, padx=10, pady=10)
    
    if saison == 'Toutes les saisons':
        choix_equipe = df_stats['Team'].unique().tolist()
    else:
        choix_equipe = df_stats[df_stats['Saison'] == saison]['Team'].unique().tolist()
    choix_equipe =  choix_equipe[:] + ['Toutes les équipes']
    equipe_combobox = ttk.Combobox(frame_menus, values=choix_equipe, state="readonly")
    equipe_combobox.set("Sélectionnez une équipe")
    equipe_combobox.grid(row=0, column=1, padx=10, pady=10)
    equipe_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_stats(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox, saison, e))


def fonction_choix_stats(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox, saison, e):
    equipe = equipe_combobox.get()

    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.39)

    label_choix_stats = tk.Label(frame_menus, text="Pour quelle stat?")
    label_choix_stats.grid(row=0, column=0, padx=10, pady=10)
    

    choix_stats =  [elem for elem in df_stats.columns if elem not in ['Saison', 'Goals Scored', 'Team', 'Opponent', 'DateTime', 'Unnamed: 0', 'Comp', 'Round', 
                                                                      'Day', 'Venue', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 
                                                                      'Notes', 'MatchID']]
    stats_combobox = ttk.Combobox(frame_menus, values=choix_stats, state="readonly")
    stats_combobox.set("Sélectionnez une stat")
    stats_combobox.grid(row=0, column=1, padx=10, pady=10)
    stats_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_filtre(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats,
                                                                                stats_combobox, saison, equipe, e))


def fonction_choix_filtre(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats, stats_combobox, saison, equipe, e):
    stats = stats_combobox.get()
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.46)

    label_choix_filtre = tk.Label(frame_menus, text="Avec quel filtre?")
    label_choix_filtre.grid(row=0, column=0, padx=10, pady=10)
    

    choix_filtre =  [elem for elem in df_stats.columns if elem in ['Saison', 'Team', 'Round', 'Opponent', 'Venue', 'Formation']]
    filtre_combobox = ttk.Combobox(frame_menus, values=choix_filtre, state="readonly")
    filtre_combobox.set("Sélectionnez un filtre")
    filtre_combobox.grid(row=0, column=1, padx=10, pady=10)
    filtre_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_type(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats, stats_combobox, 
                                                                               label_choix_filtre, filtre_combobox, saison, equipe, stats, e))

def fonction_choix_type(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats, stats_combobox, label_choix_filtre, filtre_combobox, saison, equipe, stats, e):
    filtre = filtre_combobox.get()
    
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.53)

    label_choix_type = tk.Label(frame_menus, text="Quel type de stat?")
    label_choix_type.grid(row=0, column=0, padx=10, pady=10)
    

    choix_type =  ['Somme', 'Moyenne']
    type_combobox = ttk.Combobox(frame_menus, values=choix_type, state="readonly")
    type_combobox.set("Sélectionnez un type de stats")
    type_combobox.grid(row=0, column=1, padx=10, pady=10)
    type_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_graph(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats, stats_combobox, label_choix_filtre,
                                                                              filtre_combobox,label_choix_type, type_combobox, saison, equipe, stats, filtre, e))

def fonction_choix_graph(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats, stats_combobox, label_choix_filtre,
                         filtre_combobox,label_choix_type, type_combobox, saison, equipe, stats, filtre, e):
    type = type_combobox.get()
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.6)

    label_choix_graph = tk.Label(frame_menus, text="Quel type de graph?")
    label_choix_graph.grid(row=0, column=0, padx=10, pady=10)
    

    choix_graph =  ['Linéaire', 'Barres']
    graph_combobox = ttk.Combobox(frame_menus, values=choix_graph, state="readonly")
    graph_combobox.set("Sélectionnez un graph")
    graph_combobox.grid(row=0, column=1, padx=10, pady=10)
    graph_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_affichage(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats,
                                                                             stats_combobox, label_choix_filtre, filtre_combobox,label_choix_type, 
                                                                             type_combobox,label_choix_graph, graph_combobox, saison, equipe, stats, filtre, type, e))

def fonction_affichage(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_stats,
                                                                             stats_combobox, label_choix_filtre, filtre_combobox,label_choix_type, 
                                                                             type_combobox,label_choix_graph, graph_combobox, saison, equipe, stats, filtre, type, e):
    graph = graph_combobox.get()

    label_choix_saison.destroy()
    saison_combobox.destroy()
    label_choix_equipe.destroy()
    equipe_combobox.destroy()
    label_choix_stats.destroy()
    stats_combobox.destroy()
    label_choix_filtre.destroy()
    filtre_combobox.destroy()
    label_choix_type.destroy()
    type_combobox.destroy()
    label_choix_graph.destroy()
    graph_combobox.destroy()

    if saison == 'Toutes les saisons':
        if equipe == 'Toutes les équipes':
            df_plot = df_stats
        else:   
            df_plot = df_stats[(df_stats['Team'] == str(equipe))]
    else:
        if equipe == 'Toutes les équipes':
            df_plot = df_stats[(df_stats['Saison'] == str(saison))]
        else:
            df_plot = df_stats[(df_stats['Team'] == str(equipe)) & (df_stats['Saison'] == str(saison))]
    if type == 'Somme':
        if graph == 'Linéaire':
            plot = df_plot.groupby(str(filtre))[str(stats)].sum().reset_index()

            figure = Figure(figsize = (15,6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.plot(plot[str(filtre)], plot[str(stats)], marker='o', linestyle='-')
            subplot.set_title('Évolution de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        if graph == 'Barres':
            plot = df_plot.groupby(str(filtre))[str(stats)].sum().reset_index()

            figure = Figure(figsize = (15,6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.bar(plot[str(filtre)], plot[str(stats)], color = 'blue')
            subplot.set_title('Nombre de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    else:
        if graph == 'Linéaire':
            plot = df_plot.groupby(str(filtre))[str(stats)].mean().reset_index()

            figure = Figure(figsize = (15,6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.plot(plot[str(filtre)], plot[str(stats)], marker='o', linestyle='-')
            subplot.set_title('Évolution de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        if graph == 'Barres':
            plot = df_plot.groupby(str(filtre))[str(stats)].mean().reset_index()

            figure = Figure(figsize = (15,6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.bar(plot[str(filtre)], plot[str(stats)], color = 'blue')
            subplot.set_title('Nombre de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    bouton_accueil = ttk.Button(window, text="Revenir à la page d'accueil", command=lambda: fonction_accueil([canvas.get_tk_widget()], bouton_accueil))
    bouton_accueil.pack(side=tk.BOTTOM)


def fonction_predictions(label_choix, choix_combobox, event):
    label_choix.destroy()
    choix_combobox.destroy()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

    label_choix_journée = tk.Label(frame_menus, text="Choissisez une journée")
    label_choix_journée.grid(row=0, column=0, padx=10, pady=10)

    choix_journée = ['J' + str(i) for i in affiches['Wk'].unique()]
    journée_combobox = ttk.Combobox(frame_menus, values=choix_journée, state="readonly")
    journée_combobox.set("Sélectionnez une journée")
    journée_combobox.grid(row=0, column=1, padx=10, pady=10)
    journée_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_match(label_choix_journée, journée_combobox, e))

def fonction_match(label_choix_journée, journée_combobox, event):
    journée = int(journée_combobox.get()[1:])


    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.32)

    label_choix_match = tk.Label(frame_menus, text="Choissisez un match")
    label_choix_match.grid(row=0, column=0, padx=10, pady=10)

    choix_match = [affiche for affiche, wk in zip(affiches['Home'] + ' - ' + affiches['Away'], affiches['Wk']) if wk == journée]
    match_combobox = ttk.Combobox(frame_menus, values=choix_match, state="readonly")
    match_combobox.set("Sélectionnez un match")
    match_combobox.grid(row=0, column=1, padx=10, pady=10)
    match_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_affichage_pred(label_choix_journée, journée_combobox, label_choix_match, match_combobox, journée, e))

def fonction_affichage_pred(label_choix_journée, journée_combobox,label_choix_match, match_combobox, journée, e):
    match = match_combobox.get()
    label_choix_journée.destroy()
    journée_combobox.destroy()
    label_choix_match.destroy()
    match_combobox.destroy()

    frame_tableau1 = tk.Frame(window)
    frame_tableau1.pack(expand=True, anchor="w")
    frame_tableau2 = tk.Frame(window)
    frame_tableau2.pack(expand=True, anchor="w")

    label_texte = tk.Label(frame_tableau1, text="")
    label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
    label_texte = tk.Label(frame_tableau2, text="")
    label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")

    frame_tableau1.place(in_=window, anchor="w", relx=0.17, rely=0.4)
    frame_tableau2.place(in_=window, anchor="w", relx=0.60, rely=0.4)
    
    equipe1 = match.split(' - ')[0].strip()
    equipe2 = match.split(' - ')[1].strip()

    df_match1 = fonction_tableau_stats(df_2324, first, equipe1, 'Domicile')
    df_match2 = fonction_tableau_stats(df_2324, first, equipe2, 'Extérieur')

    try:
        a = str(int(df_pred[(df_pred['Equipe 1'] == equipe1) & (df_pred['Equipe 2'] == equipe2)]['Résultat'].value_counts()[1]))
    except Exception as e:
        a = '0'
    try:
        b = str(int(df_pred[(df_pred['Equipe 1'] == equipe1) & (df_pred['Equipe 2'] == equipe2)]['Résultat'].value_counts()[0]))
    except Exception as e:
        b = '0'
    try:
        c = str(int(df_pred[(df_pred['Equipe 1'] == equipe1) & (df_pred['Equipe 2'] == equipe2)]['Résultat'].value_counts()[-1]))
    except Exception as e:
        c = '0'

    df_match1['Historique'] = a + 'V ' + b + 'N ' + c + 'D'
    df_match2['Historique'] = c + 'V ' + b + 'N ' + a + 'D'

    for i, (col_name, value) in enumerate(df_match1.iloc[0].items()):
        if col_name == 'Classement':
            value = int(value)
        label_title = tk.Label(frame_tableau1, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau1, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")

    for i, (col_name, value) in enumerate(df_match2.iloc[0].items()):
        if col_name == 'Classement':
            value = int(value)
        label_title = tk.Label(frame_tableau2, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau2, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")


    image_path1 = trouver_chemins_images_avec_mot_cle('Logos\\', equipe1)
    image1 = Image.open(image_path1)
    image1 = image1.resize((100, 120), Image.ANTIALIAS)  
    photo1 = ImageTk.PhotoImage(image1)

    label_image1 = tk.Label(frame_tableau1, image=photo1)
    label_image1.image = photo1  
    label_image1.grid(row=0, column=0, columnspan=2)

    image_path2 = trouver_chemins_images_avec_mot_cle('Logos\\', equipe2)
    image2 = Image.open(image_path2)
    image2 = image2.resize((100, 120), Image.ANTIALIAS)  
    photo2 = ImageTk.PhotoImage(image2)

    label_image2 = tk.Label(frame_tableau2, image=photo2)
    label_image2.image = photo2  
    label_image2.grid(row=0, column=0, columnspan=2)

    bouton_predictions = tk.Button(window, text="Prédictions", command=lambda: fonction_predictions_final(equipe1, equipe2, frame_tableau1, frame_tableau2, bouton_predictions, event = None))
    bouton_predictions.place(relx=0.5, rely=0.8, anchor="center")

def fonction_predictions_final(equipe1, equipe2, frame_tableau1, frame_tableau2, bouton_prediction, event):
    bouton_prediction.destroy()
    frame_tableau1.destroy()
    frame_tableau2.destroy()
    bouton_prediction.destroy()
    
    frame_tableau1 = tk.Frame(window)
    frame_tableau1.pack(expand=True, anchor="w")
    frame_tableau2 = tk.Frame(window)
    frame_tableau2.pack(expand=True, anchor="w")

    image_path1 = trouver_chemins_images_avec_mot_cle('Logos\\', equipe1)
    image1 = Image.open(image_path1)
    image1 = image1.resize((100, 120), Image.ANTIALIAS)  
    photo1 = ImageTk.PhotoImage(image1)

    label_image1 = tk.Label(frame_tableau1, image=photo1)
    label_image1.image = photo1  
    label_image1.grid(row=0, column=0, columnspan=2)

    image_path2 = trouver_chemins_images_avec_mot_cle('Logos\\', equipe2)
    image2 = Image.open(image_path2)
    image2 = image2.resize((100, 120), Image.ANTIALIAS)  
    photo2 = ImageTk.PhotoImage(image2)

    label_image2 = tk.Label(frame_tableau2, image=photo2)
    label_image2.image = photo2  
    label_image2.grid(row=0, column=0, columnspan=2)

    frame_tableau1.place(in_=window, anchor="w", relx=0.37, rely=0.4)
    frame_tableau2.place(in_=window, anchor="w", relx=0.57, rely=0.4)
    
    X_test = df_2324[(df_2324['Equipe 1'] == equipe1) & (df_2324['Journée'] == first)].drop(['Saison', 'Journée', 'Résultat', 'Equipe 1', 'Equipe 2', 'Buts Equipe 1', 'Buts Equipe 2'], axis=1)
    Y_pred_1 = model1.predict(X_test)
    Y_pred_2 = model2.predict(X_test)
    resultat = Y_pred_1 - Y_pred_2

    bouton_accueil = ttk.Button(window, text="Revenir à la page d'accueil", command=lambda: fonction_accueil([frame_tableau1, frame_tableau2], bouton_accueil))
    bouton_accueil.pack(side=tk.BOTTOM)

def fonction_stats(label_choix, choix_combobox, event):
    label_choix.destroy()
    choix_combobox.destroy()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)
    label_choix_saison = tk.Label(frame_menus, text="Choisissez une saison")
    label_choix_saison.grid(row=0, column=0, padx=10, pady=10)

    choix_saisons = df_stats['Saison'].unique().tolist()
    choix_saisons =  choix_saisons[:]
    saison_combobox = ttk.Combobox(frame_menus, values=choix_saisons, state="readonly")
    saison_combobox.set("Sélectionnez une saison")
    saison_combobox.grid(row=0, column=1, padx=10, pady=10)
    saison_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_equipe_stats(label_choix_saison, saison_combobox, e))

def fonction_choix_equipe_stats(label_choix_saison, saison_combobox, e):
    saison = saison_combobox.get()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.32)

    label_choix_equipe = tk.Label(frame_menus, text="Choisissez une équipe")
    label_choix_equipe.grid(row=0, column=0, padx=10, pady=10)
    

    choix_equipe = df_stats[df_stats['Saison'] == saison]['Team'].unique().tolist()
    choix_equipe =  choix_equipe[:]
    equipe_combobox = ttk.Combobox(frame_menus, values=choix_equipe, state="readonly")
    equipe_combobox.set("Sélectionnez une équipe")
    equipe_combobox.grid(row=0, column=1, padx=10, pady=10)
    equipe_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_journee_stats(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox, saison, e))

def fonction_choix_journee_stats(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox, saison, e):
    equipe = equipe_combobox.get()
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.39)

    label_choix_journee = tk.Label(frame_menus, text="Choisissez une journée")
    label_choix_journee.grid(row=0, column=0, padx=10, pady=10)
    

    choix_journee = df_stats[(df_stats['Saison'] == saison) & (df_stats['Team'] == equipe)]['Round'].unique().tolist()
    choix_journee =  choix_journee[:]
    journee_combobox = ttk.Combobox(frame_menus, values=choix_journee, state="readonly")
    journee_combobox.set("Sélectionnez une journée")
    journee_combobox.grid(row=0, column=1, padx=10, pady=10)
    journee_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_affichage_stats(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_journee, journee_combobox, saison, equipe, e))

def fonction_affichage_stats(label_choix_saison, saison_combobox, label_choix_equipe, equipe_combobox,label_choix_journee, journee_combobox, saison, equipe, e):
    journee = journee_combobox.get()
    label_choix_saison.destroy()
    saison_combobox.destroy()
    label_choix_equipe.destroy()
    equipe_combobox.destroy()
    label_choix_journee.destroy()
    journee_combobox.destroy()

    df_stats_unique = df_stats[(df_stats['Saison'] == saison) & (df_stats['Team'] == equipe) & (df_stats['Round'] == int(journee))].drop(['Goals Scored', 'Victoire', 'Défaite', 'Unnamed: 0', 'Notes', 'Match Report', 'Nul', 'DateTime', 'Saison','MatchID', 'Team', 'Comp'] , axis=1).copy()

    frame_tableau1 = tk.Frame(window)
    frame_tableau1.pack(expand=True, anchor="w")
    label_texte = tk.Label(frame_tableau1, text="")
    label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
    frame_tableau1.place(in_=window, anchor="w", relx=0.03, rely=0.58)

    for i, (col_name, value) in enumerate(df_stats_unique.iloc[0, :28].iteritems()):
        label_title = tk.Label(frame_tableau1, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau1, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")
    

    frame_tableau2 = tk.Frame(window)
    frame_tableau2.pack(expand=True, anchor="w")
    label_texte = tk.Label(frame_tableau2, text="")
    label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
    frame_tableau2.place(in_=window, anchor="w", relx=0.28, rely=0.58)

    for i, (col_name, value) in enumerate(df_stats_unique.iloc[0, 28:56].iteritems()):
        label_title = tk.Label(frame_tableau2, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau2, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")
    
    frame_tableau3 = tk.Frame(window)
    frame_tableau3.pack(expand=True, anchor="w")
    label_texte = tk.Label(frame_tableau2, text="")
    label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
    frame_tableau3.place(in_=window, anchor="w", relx=0.53, rely=0.6)

    for i, (col_name, value) in enumerate(df_stats_unique.iloc[0, 56:84].iteritems()):
        label_title = tk.Label(frame_tableau3, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau3, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")
    
    frame_tableau4 = tk.Frame(window)
    frame_tableau4.pack(expand=True, anchor="w")
    label_texte = tk.Label(frame_tableau2, text="")
    label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
    frame_tableau4.place(in_=window, anchor="w", relx=0.78, rely=0.6)

    for i, (col_name, value) in enumerate(df_stats_unique.iloc[0, 84:112].iteritems()):
        label_title = tk.Label(frame_tableau4, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau4, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")

    bouton_accueil = ttk.Button(window, text="Revenir à la page d'accueil", command=lambda: fonction_accueil([frame_tableau1, frame_tableau2, frame_tableau3, frame_tableau4], bouton_accueil))
    bouton_accueil.pack(side=tk.BOTTOM)


def toggle_fullscreen(event=None):
    state = not window.attributes('-fullscreen')
    window.attributes('-fullscreen', state)


window = tk.Tk()
window.title("Choix de l'utilisateur")
window.bind('<Escape>', toggle_fullscreen)
window.attributes('-fullscreen', True)


image_intro = Image.open('Logos\\Logo_Ligue_1_Uber_Eats_2022.png')
image_intro = image_intro.resize((130, 180), Image.LANCZOS)
photo_intro = ImageTk.PhotoImage(image_intro)
frame_intro = tk.Frame(window)
frame_intro.pack(expand=True, anchor="w")
label_image_intro = tk.Label(frame_intro, image=photo_intro)
label_image_intro.image = photo_intro
label_image_intro.grid(row=0, column=0)


frame_menus = tk.Frame(window)
frame_menus.pack(expand=True, anchor="w")
frame_intro.place(in_=window, anchor="w", relx=0.47, rely=0.1)

label_choix = tk.Label(frame_menus, text="Que voulez vous faire?")
label_choix.grid(row=0, column=0, padx=10, pady=10)

choix_combobox = ttk.Combobox(frame_menus, values=['Visualisez des stats', 'Parcourir les données', 'Prédire un résultat futur'], state="readonly")
choix_combobox.set("Sélectionnez un choix")
choix_combobox.grid(row=0, column=1, padx=10, pady=10)
choix_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix(label_choix, choix_combobox, e))

frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

window.mainloop()



In [13]:
df_pred


,Saison,Journée,Equipe 1,Equipe 2,Equipe 1 à Domicile,Buts Equipe 1,Buts Equipe 2,Résultat,Classement Equipe 1,Classement Equipe 2,...,Moyenne_BM par 2 à Extérieur,Moyenne_BE par 1,Moyenne_BE par 2,Moyenne_BE par 1 à Domicile,Moyenne_BE par 2 à Domicile,Moyenne_BE par 1 à Extérieur,Moyenne_BE par 2 à Extérieur,Forme 1,Forme 2,Historique
0,2002-2003,1,Ajaccio,Strasbourg,0,1,1,0,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2002-2003,2,Ajaccio,Guingamp,1,0,2,-1,9.0,7.0,...,NaN,1.000000,3.000000,NaN,3.000000,1.000000,NaN,0.0,0.0,0
2,2002-2003,3,Ajaccio,Paris S-G,0,2,2,0,16.0,6.0,...,0.000000,1.500000,0.000000,2.000000,0.000000,1.000000,0.000000,-1.0,1.0,0
3,2002-2003,4,Ajaccio,Sedan,1,1,0,1,15.0,19.0,...,1.000000,1.666667,2.333333,2.000000,0.500000,1.500000,6.000000,-1.0,-2.0,0
4,2002-2003,5,Ajaccio,Troyes,0,0,1,-1,13.0,17.0,...,0.000000,1.250000,1.500000,1.000000,3.000000,1.500000,0.000000,0.0,-2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2023-2024,13,Toulouse,Nice,0,0,1,-1,15.0,2.0,...,1.166667,1.333333,0.333333,1.200000,0.166667,1.428571,0.500000,-2.0,4.0,0
319,2023-2024,14,Toulouse,Lorient,1,1,1,0,15.0,16.0,...,1.000000,1.307692,1.846154,1.200000,1.571429,1.375000,2.166667,-3.0,-2.0,0
320,2023-2024,15,Toulouse,Lyon,0,0,3,-1,15.0,18.0,...,0.571429,1.285714,1.928571,1.166667,2.285714,1.375000,1.571429,-3.0,-2.0,0
321,2023-2024,16,Toulouse,Rennes,1,0,0,0,15.0,13.0,...,0.333333,1.400000,1.400000,1.166667,1.555556,1.555556,1.166667,-3.0,-3.0,0
